# Azure Cognitive Search sample 
## Passing Images as Binary File References

Cognitive Search skillsets that need to pass images to custom skills use a binary file reference to serialize the images to pass them to and from skills. This sample demonstrates an example of how skills can be configured to accept an image as an input from the skillset and return images as outputs to the skillset. This example does nothing more than segment an image based on the layout from OCR. The sole purpose of this sample is to demonstrate how you pass images to skills and how skills can return images.



### Prerequisites 

Provision the required services:
1. [Azure Cognitive Search](https://docs.microsoft.com/azure/search/search-create-service-portal)
2. [Azure Functions](https://docs.microsoft.com/azure/azure-functions/) used for hosting an API endpoint.
3. [Storage Account](https://docs.microsoft.com/azure/storage/blobs/)


### Deploy the Azure functions app 
The ```SplitImage``` folder contains an Azure function that will accept an input in the [custom skill format](https://docs.microsoft.com/azure/search/cognitive-search-custom-skill-web-api#skill-inputs). 
Each input record contains an image that is serialized as a ```Base64``` encoded string and the layout text returned from the OCR skill.
The skill then segments the image into smaller images based on the coordinates of the layout text. It then returns a list of images, each ```Base64``` encoded back to the skillset. While this is not very useful, you could build a [Custom Vision](https://github.com/Azure-Samples/azure-search-power-skills/tree/master/Vision/CustomVision) skill to perform a useful inference on your images.

Follow the [Azure Functions tutorial](https://docs.microsoft.com/azure/developer/python/tutorial-vs-code-serverless-python-05) to deploy the function. Once the deployment completes, navigate to the function app in the portal, select the function (SplitImage) and click the Get Function Url button. Save the function url as we will use it in the next step.

### Create the enrichment pipeline
In the next few steps we will configure the Cognitive Search enrichment pipeline with the following steps:
1. Create a blob storage data source. Ensure you have a blob storage container with at least one file containing images.
2. Create a skillset to enrich the documents in the data source
3. Create an index
4. Create an indexer to move documents from the data source to the index while invoking the skillset


In [ ]:
!pip install azure-storage-blob

import os
import json
import requests

# Configure all required variables for Cognitive Search. Replace each with the credentials from your accounts.

# Replace with Search Service name, API key, and endpoint from the Azure portal.
search_service = "" # In the format "https://searchservicename.search.windows.net"
api_key = 'your search service API key'

# Leave the API version and content_type as they are listed here.
api_version = '2020-06-30'
content_type = 'application/json'

# Replace with a Cognitive Services all in one key.
cog_svcs_key = '' #Required only if processing more than 20 documents
cog_svcs_acct = 'your cog services account name'

#Connection string to the storage account. This will be used for the datasource, knowledge store and cache
STORAGECONNSTRING = "DefaultEndpointsProtocol=https;AccountName=<Storage Acct>;AccountKey=<KEY>;EndpointSuffix=core.windows.net"
# The container with your files containing images
datasource_container = 'bfrsample' # Replace with the container containging your files
# This sample assumes you will use the same storage account for the datasource, knowledge store and indexer cache. The knowledge store will contain the projected images
know_store_cache = STORAGECONNSTRING
# Container where the sliced images will be projected to
know_store_container = "obfuscated"
skill_uri = "https://<skillname>.azurewebsites.net/api/SplitImage?code=CODE"

Create a helper function to invoke the Cognitive Search REST API

In [ ]:
def construct_Url(service, resource, resource_name, action, api_version):
    if resource_name:
        
        if action:
            return service + '/'+ resource + '/' + resource_name + '/' + action + '?api-version=' + api_version
        else:
            return service + '/'+ resource + '/' + resource_name + '?api-version=' + api_version
    else:
        return service + '/'+ resource + '?api-version=' + api_version


headers = {'api-key': api_key, 'Content-Type': content_type}
# Test out the URLs to ensure that the configuration works
print(construct_Url(search_service, "indexes", "bfr-sample", "analyze", api_version))
print(construct_Url(search_service, "indexes", "bfr-sample", None, api_version))
print(construct_Url(search_service, "indexers", None, None, api_version))

#### Create the data source

In [ ]:
container = datasource_container

datsource_def = {
    'name': f'{datasource_container}-ds',
    'description': f'Datasource containing files with sample images',
    'type': 'azureblob',
    'subtype': None,
    'credentials': {
        'connectionString': f'{STORAGECONNSTRING}'
    },
    'container': {
        'name': f'{datasource_container}'
    },

}
r = requests.post(construct_Url(search_service, "datasources", None, None, api_version), data=json.dumps(datsource_def),  headers=headers)
print(r)
res = r.json()
print(json.dumps(res, indent=2))

#### Create the skillset

In [ ]:
skillset_name = f'{datasource_container}-ss'
skillset_def = {
    'name': f'{skillset_name}',
    'description': 'Skillset to enrich hotel reviews with aspect based sentiment',
    'skills': [
        {
          "@odata.type": "#Microsoft.Skills.Vision.OcrSkill",
          "name": "OCRSkill",
          "description": "OCR Skill",
          "context": "/document/normalized_images/*",
          "textExtractionAlgorithm": None,
          "lineEnding": "Space",
          "defaultLanguageCode": "en",
          "detectOrientation": True,
          "inputs": [
            {
              "name": "image",
              "source": "/document/normalized_images/*"
            }
          ],
          "outputs": [
            {
              "name": "text",
              "targetName": "text"
            },
            {
              "name": "layoutText",
              "targetName": "layoutText"
            }
          ]
        },
        {
          "@odata.type": "#Microsoft.Skills.Text.PIIDetectionSkill",
          "name": "#1",
          "description": "",
          "context": "/document/merged_content",
          "defaultLanguageCode": "en",
          "minimumPrecision": 0.5,
          "maskingMode": "replace",
          "maskingCharacter": "*",
          "inputs": [
            {
              "name": "text",
              "source": "/document/merged_content"
            }
          ],
          "outputs": [
            {
              "name": "piiEntities",
              "targetName": "pii_entities"
            },
            {
              "name": "maskedText",
              "targetName": "masked_text"
            }
          ]
        },
        {
          "@odata.type": "#Microsoft.Skills.Custom.WebApiSkill",
          "name": "ImageSkill",
          "description": "Segment Images",
          "context": "/document/normalized_images/*",
          "uri": f'{skill_uri}',
          "httpMethod": "POST",
          "timeout": "PT30S",
          "batchSize": 1000,
          "degreeOfParallelism": 1,
          "inputs": [
            {
              "name": "image",
              "source": "/document/normalized_images/*"
            },
            {
              "name": "layoutText",
              "source": "/document/normalized_images/*/layoutText"
            },
            {
              "name": "pii_entities",
              "source": "/document/merged_content/pii_entities"
            }
          ],
          "outputs": [
            {
              "name": "slices",
              "targetName": "slices"
            },
            {
              "name": "original",
              "targetName": "original"
            }
          ],
          "httpHeaders": {}
        },
        {
          "@odata.type": "#Microsoft.Skills.Text.MergeSkill",
          "name": "MergeSkill",
          "description": "Merge results from cracking with OCR text",
          "context": "/document",
          "insertPreTag": " ",
          "insertPostTag": " ",
          "inputs": [
            {
              "name": "text",
              "source": "/document/content"
            },
            {
              "name": "itemsToInsert",
              "source": "/document/normalized_images/*/text"
            },
            {
              "name": "offsets",
              "source": "/document/normalized_images/*/contentOffset"
            }
          ],
          "outputs": [
            {
              "name": "mergedText",
              "targetName": "merged_content"
            }
          ]
        }
        
    ],
    'cognitiveServices':None,
    'knowledgeStore': {
        'storageConnectionString': f'{STORAGECONNSTRING}',
        'projections': [
          {
        "tables": [],
        "objects": [
          {
            "storageContainer": "layout",
            "referenceKeyName": None,
            "generatedKeyName": "layoutKey",
            "source": "/document/normalized_images/*/layoutText",
            "sourceContext": None,
            "inputs": []
          }
        ],
        "files": [
          {
            "storageContainer": "slices",
            "referenceKeyName": None,
            "generatedKeyName": "slicesKey",
            "source": "/document/normalized_images/*/slices/*",
            "sourceContext": None,
            "inputs": []
          },
          {
            "storageContainer": "images",
            "referenceKeyName": None,
            "generatedKeyName": "imageKey",
            "source": "/document/normalized_images/*",
            "sourceContext": None,
            "inputs": []
          },
          {
            "storageContainer": f'{know_store_container}',
            "referenceKeyName": None,
            "generatedKeyName": "originalKey",
            "source": "/document/normalized_images/*/original",
            "sourceContext": None,
            "inputs": []
          }
        ]
      }
        ]
    }
}


r = requests.put(construct_Url(search_service, "skillsets", skillset_name, None, api_version), data=json.dumps(skillset_def),  headers=headers)
print(r)

res = r.json()
print(json.dumps(res, indent=2))

#### Create the index

In [ ]:
indexname = f'{datasource_container}-idx'
index_def = {
    "name":f'{indexname}',
      "defaultScoringProfile": "",
    "fields": [
        {
          "name": "image_text",
          "type": "Collection(Edm.String)",
          "facetable": False,
          "filterable": False,
          "retrievable": True,
          "searchable": True,
          "analyzer": "standard.lucene",
          "indexAnalyzer": None,
          "searchAnalyzer": None,
          "synonymMaps": [],
          "fields": []
        },
  
        {
          "name": "content",
          "type": "Edm.String",
          "facetable": False,
          "filterable": False,
          "key": False,
          "retrievable": True,
          "searchable": True,
          "sortable": False,
          "analyzer": "standard.lucene",
          "indexAnalyzer": None,
          "searchAnalyzer": None,
          "synonymMaps": [],
          "fields": []
        },
        {
            "name": "metadata_storage_content_type",
            "type": "Edm.String",
            "searchable": False,
            "filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": False,
            "key": False,
            "indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": None,
            "synonymMaps": []
        },
        {
            "name": "metadata_storage_size",
            "type": "Edm.Int64",
            "searchable": False,
            "filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": False,
            "key": False,
            "indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": None,
            "synonymMaps": []
        },
        {
            "name": "metadata_storage_last_modified",
            "type": "Edm.DateTimeOffset",
            "searchable": False,
            "filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": False,
            "key": False,
            "indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": None,
            "synonymMaps": []
        },
        {
            "name": "metadata_storage_content_md5",
            "type": "Edm.String",
            "searchable": False,
            "filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": False,
            "key": False,
            "indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": None,
            "synonymMaps": []
        },
        {
            "name": "metadata_storage_name",
            "type": "Edm.String",
            "searchable": False,
            "filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": False,
            "key": False,
            "indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": None,
            "synonymMaps": []
        },
        {
            "name": "metadata_storage_path",
            "type": "Edm.String",
            "searchable": False,
            "filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": False,
            "key": True,
            "indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": None,
            "synonymMaps": []
        },
        {
            "name": "metadata_storage_file_extension",
            "type": "Edm.String",
            "searchable": False,
            "filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": False,
            "key": False,
            "indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": None,
            "synonymMaps": []
        }
        
    ],
    "scoringProfiles": [],
    "corsOptions": None,
    "suggesters": [
        {
            "name": "sg",
            "searchMode": "analyzingInfixMatching",
            "sourceFields": [
                "metadata_storage_path"
            ]
        }
    ],
    "analyzers": [],
    "tokenizers": [],
    "tokenFilters": [],
    "charFilters": [],
    "encryptionKey": None,
    "similarity": None
}
r = requests.post(construct_Url(search_service, "indexes", None, None, api_version), data=json.dumps(index_def),  headers=headers)
print(r)
res = r.json()
print(json.dumps(res, indent=2))

#### Create the indexer

In [ ]:
indexername = f'{datasource_container}-idxr'
indexer_def = {
    "name": f'{indexername}',
    "description": "Indexer to enrich hotel reviews",
    "dataSourceName": f'{datasource_container}-ds',
    "skillsetName": f'{datasource_container}-ss',
    "targetIndexName": f'{datasource_container}-idx',
    "disabled": None,
    "schedule": {
        "interval": "PT2H",
        "startTime": "0001-01-01T00:00:00Z"
      },
    "parameters": {
    "batchSize": None,
    "maxFailedItems": 0,
    "maxFailedItemsPerBatch": 0,
    "base64EncodeKeys": None,
    "configuration": {
      "dataToExtract": "contentAndMetadata",
      "parsingMode": "default",
      "imageAction": "generateNormalizedImages"
        }
    },
   "fieldMappings": [
    {
      "sourceFieldName": "metadata_storage_path",
      "targetFieldName": "metadata_storage_path",
      "mappingFunction": {
        "name": "base64Encode"
      }
    }
    ],
    "outputFieldMappings": [
        {
          "sourceFieldName": "/document/normalized_images/*/text",
          "targetFieldName": "image_text"
        }
    ],
    "cache": {
        "enableReprocessing": True,
        "storageConnectionString": f'{know_store_cache}'
    }
}
r = requests.post(construct_Url(search_service, "indexers", None, None, api_version), data=json.dumps(indexer_def), headers=headers)
print(r)
res = r.json()
print(json.dumps(res, indent=2))

#### Run the indexer

In [ ]:
r = requests.post(construct_Url(search_service, "indexers", indexername, "run", api_version), data=None,  headers=headers)
print(r)
#res = r.json()
#print(json.dumps(res, indent=2))

### View Results
The following cell downloads the image so that you can verify skillset success.

In [1]:
from IPython.display import Image
import base64
from azure.storage.blob import ContainerClient
count = 0
container = ContainerClient.from_connection_string(conn_str=STORAGECONNSTRING, container_name=know_store_container)
blob_list = container.list_blobs()
for blob in blob_list:
    print(blob.name + '\n')
    blob_client = container.get_blob_client(blob.name)
    with open("image" + str(count) + ".jpg", "wb") as my_blob:
                download_stream = blob_client.download_blob()
                my_blob.write(download_stream.readall())
    count = count + 1
    if(count == 3):
        break

Image(filename='image2.jpg') 

NameError: name 'STORAGECONNSTRING' is not defined

### Next Steps
You now know how to pass images into skills and even return images to the skillset. As a next step, you can start from scratch and build a [custom AML Skill](https://docs.microsoft.com/en-us/azure/search/cognitive-search-aml-skill) to perform inferences on images or use the Custom Vision service to build a skill. The power skills github repository has a [sample custom vision skill](https://github.com/Azure-Samples/azure-search-power-skills/tree/master/Vision/CustomVision) to help you get started.